# Set up

In [1]:
# update path with data dir
import sys
sys.path.append('../data/')
sys.path.append('../modelling/')

In [2]:
import player_data as player
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import patsy
from datetime import datetime
import numpy as np

/home/tom/footbot/venv/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
response_variable = 'goals_scored'

# Getting data

In [17]:
file_name = 'csvs/element_gameweek_2019-05-14.csv'

In [18]:
try:
    element_gameweek_df = pd.read_csv(file_name)
except:
    element_gameweek_df = player.get_element_gameweek_df()
    element_gameweek_df.to_csv(file_name)

In [19]:
current_event = 32
element_types = [2, 3, 4]
threshold_minutes = 1

In [20]:
element_gameweek_df =\
element_gameweek_df[
    (element_gameweek_df['event'] <= current_event)
    & (element_gameweek_df['element_type'].isin(element_types))
    & (element_gameweek_df['minutes'] >= threshold_minutes)
]

In [21]:
player.add_home_categorical_variable(element_gameweek_df)

In [22]:
element_gameweek_df['row_id'] = element_gameweek_df['row_id'].rank()

In [23]:
element_gameweek_df['element_fixture_rank'] = element_gameweek_df.groupby('element')['row_id'].rank()

In [24]:
rolling_cols = [
    'big_chances_missed',
    'goals_scored',
    'target_missed'
]

In [25]:
rolling_df = \
element_gameweek_df.copy().groupby('element', as_index=True)[['minutes'] + rolling_cols]\
.rolling(38, min_periods=1).sum().reset_index()[['element', 'minutes'] + rolling_cols]

In [26]:
rolling_df['element_fixture_rank'] = rolling_df.groupby('element')['minutes'].rank()

In [27]:
rolling_df['element_fixture_rank'] = rolling_df['element_fixture_rank'] + 1

In [28]:
element_gameweek_df =\
element_gameweek_df.join(
    rolling_df.set_index(['element', 'element_fixture_rank']),
    on=['element', 'element_fixture_rank'],
    rsuffix='_rolling')

In [29]:
for i in rolling_cols:
    element_gameweek_df[i + '_per_minute_rolling'] = \
    element_gameweek_df[i + '_rolling'] / element_gameweek_df['minutes_rolling']

In [30]:
rolling_p5_df = \
element_gameweek_df.copy().groupby('element', as_index=True)[['minutes'] + rolling_cols]\
.rolling(5, min_periods=1).sum().reset_index()[['element', 'minutes'] + rolling_cols]

In [31]:
rolling_p5_df['element_fixture_rank'] = rolling_p5_df.groupby('element')['minutes'].rank()

In [32]:
rolling_p5_df['element_fixture_rank'] = rolling_p5_df['element_fixture_rank'] + 1

In [33]:
element_gameweek_df =\
element_gameweek_df.join(
    rolling_p5_df.set_index(['element', 'element_fixture_rank']),
    on=['element', 'element_fixture_rank'],
    rsuffix='_rolling_p5')

In [34]:
for i in rolling_cols:
    element_gameweek_df[i + '_per_minute_rolling_p5'] = \
    element_gameweek_df[i + '_rolling_p5'] / element_gameweek_df['minutes_rolling']

In [35]:
element_gameweek_df.head()

,Unnamed: 0,row_id,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,...,big_chances_missed_per_minute_rolling,goals_scored_per_minute_rolling,target_missed_per_minute_rolling,minutes_rolling_p5,big_chances_missed_rolling_p5,goals_scored_rolling_p5,target_missed_rolling_p5,big_chances_missed_per_minute_rolling_p5,goals_scored_per_minute_rolling_p5,target_missed_per_minute_rolling_p5
92,92,1.0,0,92,0,0,0,11,0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,94,2.0,0,74,0,0,0,17,0,4,...,0.0,0.0,0.011111,90.0,0.0,0.0,1.0,0.0,0.0,0.011111
95,95,3.0,0,38,0,0,0,10,0,2,...,0.0,0.0,0.006289,159.0,0.0,0.0,1.0,0.0,0.0,0.006289
96,96,4.0,0,64,0,0,0,19,0,10,...,0.0,0.0,0.004902,204.0,0.0,0.0,1.0,0.0,0.0,0.004902
97,97,5.0,0,89,0,0,0,17,0,2,...,0.0,0.0,0.003401,294.0,0.0,0.0,1.0,0.0,0.0,0.003401


In [36]:
element_gameweek_df.columns

Index(['Unnamed: 0', 'row_id', 'assists', 'attempted_passes',
       'big_chances_created', 'big_chances_missed', 'bonus', 'bps',
       'clean_sheets', 'clearances_blocks_interceptions', 'completed_passes',
       'creativity', 'dribbles', 'ea_index', 'element',
       'errors_leading_to_goal', 'errors_leading_to_goal_attempt', 'fixture',
       'fouls', 'goals_conceded', 'goals_scored', 'ict_index', 'id',
       'influence', 'key_passes', 'kickoff_time', 'kickoff_time_formatted',
       'loaned_in', 'loaned_out', 'minutes', 'offside', 'open_play_crosses',
       'opponent_team', 'own_goals', 'penalties_conceded', 'penalties_missed',
       'penalties_saved', 'recoveries', 'red_cards', 'round', 'saves',
       'selected', 'tackled', 'tackles', 'target_missed', 'team_a_score',
       'team_h_score', 'threat', 'total_points', 'transfers_balance',
       'transfers_in', 'transfers_out', 'value', 'was_home', 'winning_goals',
       'yellow_cards', 'event', 'team_a', 'team_a_difficulty', '

# Neural network

In [37]:
def add_missing_columns(df, columns):
    for col in set(columns) - set(df.columns):
        df[col] = np.zeros(len(df))
    
    return df[columns]

In [38]:
rolling_vars = ' + '.join([i + '_per_minute_rolling' for i in rolling_cols])
rolling_p5_vars = ' + '.join([i + '_per_minute_rolling_p5' for i in rolling_cols])

formula = \
f'{response_variable} ~ C(element) + C(opposition_team) + was_home'

print(formula)

goals_scored ~ C(element) + C(opposition_team) + was_home


In [39]:
y, X = patsy.dmatrices(formula, element_gameweek_df, return_type='dataframe')

In [40]:
len(X.columns)

479

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [42]:
X_train = add_missing_columns(X_train, X.columns)
X_test = add_missing_columns(X_test, X.columns)

In [43]:
# standardise explanatory data by subtracting the mean and scaling to unit variance
# find standardising transformations based on training set
scaler = StandardScaler().fit(X_train)

# standardise training set
X_train = scaler.transform(X_train)

# standardise test set
X_test = scaler.transform(X_test)

In [44]:
# instantiate model
model = Sequential()

# add the input layer, i.e. first hidden layer
model.add(Dense(units=25, activation='relu', input_shape=(len(X.columns),)))
          
# add the output layer
model.add(Dense(units=1, activation='linear'))

Instructions for updating:
Colocations handled automatically by placer.


In [45]:
# set training configuration
model.compile(
    loss='mean_absolute_error',
    optimizer='adam'
)

In [46]:
# train model on training set
model.fit(
    X_train, # explanatory variable training data
    y_train, # response variable training data
    epochs=20, # number of training iterations
    batch_size=10, # investigate this - The batch size that you specify in the code above defines the number of samples that going to be propagated through the network
    verbose=1 # monitor training progress
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
7205/7205 [==============================] - 1s 182us/sample - loss: 0.3962
Epoch 2/20
7205/7205 [==============================] - 1s 136us/sample - loss: 0.1751
Epoch 3/20
7205/7205 [==============================] - 1s 138us/sample - loss: 0.1356
Epoch 4/20
7205/7205 [==============================] - 1s 135us/sample - loss: 0.1176
Epoch 5/20
7205/7205 [==============================] - 1s 141us/sample - loss: 0.1105
Epoch 6/20
7205/7205 [==============================] - 1s 139us/sample - loss: 0.1090
Epoch 7/20
7205/7205 [==============================] - 1s 137us/sample - loss: 0.1088
Epoch 8/20
7205/7205 [==============================] - 1s 136us/sample - loss: 0.1085
Epoch 9/20
7205/7205 [==============================] - 1s 136us/sample - loss: 0.1083
Epoch 10/20
7205/7205 [==============================] - 1s 136us/sample - loss: 0.1083
Epoch 11/20
7205/7205 [==============================] - 1s 137us/sample - loss: 

In [47]:
# get predictions for test set
y_pred = model.predict(X_test).flatten()

# calculate cross entropy loss for test set
test_loss = (mean_absolute_error(y_test, y_pred), mean_squared_error(y_test, y_pred))
test_loss

(0.10198090648326079, 0.12193335305954586)